# Preparation

Install my forck of the  *shap* package:

(The PyTorchDeepExplainer from the official master branch needs some tweaking to work)

In [1]:
# !pip install https://github.com/ceshine/shap/archive/master.zip

In [2]:
import sys
import gc

In [3]:
import torch.nn as nn
import torch
from torch.nn.utils.weight_norm import weight_norm
from sklearn import preprocessing
import shap
import numpy as np
import joblib

## Load a Model

In [4]:
%ls ../input/data/cache/model_cache/

ls: ../input/data/cache/model_cache/: No such file or directory


In [5]:
MODEL = "../input/data/cache/model_cache/snapshot_PUBG_0.02873547.pth"

In [6]:
class MLPModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            weight_norm(nn.Linear(num_features, 64)),
            nn.ELU(),
            weight_norm(nn.Linear(64, 64)),
            nn.ELU(),
            weight_norm(nn.Linear(64, 64)),
            nn.ELU(),
            weight_norm(nn.Linear(64, 64)),
            nn.ELU(),          
            weight_norm(nn.Linear(64, 1)),
        )

    def forward(self, input_tensor):
        return torch.clamp(self.model(input_tensor), 0, 1)

In [7]:
x_train, features = joblib.load("../input/x_train_dump.jl")

FileNotFoundError: [Errno 2] No such file or directory: '../input/x_train_dump.jl'

In [ ]:
DEVICE = "cpu"
model = MLPModel(len(features)).to(DEVICE)
model.load_state_dict(torch.load(MODEL, map_location='cpu'))

## Deep Explainer

Here we only use a small sample (300) to save time:

In [8]:
%%time
e = shap.DeepExplainer(
        model, 
        torch.from_numpy(
            x_train[np.random.choice(np.arange(len(x_train)), 10000, replace=False)]
        ).to(DEVICE))

NameError: name 'model' is not defined

In [9]:
%%time
x_samples = x_train[np.random.choice(np.arange(len(x_train)), 300, replace=False)]
print(len(x_samples))
shap_values = e.shap_values(
    torch.from_numpy(x_samples).to(DEVICE)
)

NameError: name 'x_train' is not defined

In [ ]:
shap_values.shape

### Shap Values As a Data Frame

In [ ]:
import pandas as pd
df = pd.DataFrame({
    "mean_abs_shap": np.mean(np.abs(shap_values), axis=0), 
    "stdev_abs_shap": np.std(np.abs(shap_values), axis=0), 
    "name": features
})
df.sort_values("mean_abs_shap", ascending=False)[:10]

### Plotting Overall Shap Values

In [ ]:
shap.summary_plot(shap_values, features=x_samples, feature_names=features)